In [ ]:
## Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
## Spatial Packages
import geopandas as gpd
from pylab import figure, scatter, show
import pysal as ps
from splot.libpysal import plot_spatial_weights
import libpysal
import mapclassify as mc
import esda 

In [ ]:
def get_data():
    
    food = pd.read_csv('data/food_all.csv', encoding='latin-1')
    food_sum = food.groupby('ccode')['pt_attempt', 'pt_suc'].sum().reset_index()
    
    
    world = gpd.read_file('data/worldmap/cshapes.shp')
    world = world.loc[world.COWEYEAR == 2016]
    
    base = world.merge(food_sum, left_on='COWCODE', right_on='ccode', how='left')
    
    region = pd.read_csv('data/codeswregions.csv')
    
    base = base.merge(region, on='ccode', how='left')
    base['pt_attempt'].fillna(0, inplace=True)
    base['pt_suc'].fillna(0, inplace=True)
    return base

df = get_data()

Lets make some basic weights with the four basic options.

1) **Contiguity** - refers to the shared vertice between two polygons.
    * Queen - When the shared vertice has a lower threshold. Essentially any points shared.
    * Rook - More restrictive.  The vertice must be of some minimum threshold for a spatial weight to be applied.
2) **K nearest Neighbors** - Set a threshold of nearest neighbors which are given equal spatial weights.

3) **Kernel Based** - bandwith

In [ ]:
def basic_weights(shape, kernel_band = 20, kn = 4):
    
    queen = ps.lib.weights.contiguity.Queen.from_dataframe(shape)

    rook = ps.lib.weights.contiguity.Rook.from_dataframe(shape)
    
    kernel = ps.lib.weights.Kernel.from_dataframe(shape, bandwidth=kernel_band)
    
    knn = ps.lib.weights.KNN.from_dataframe(shape, k=kn)
    
    return queen, rook, kernel, knn

queen, rook, kernel, knn = basic_weights(df)

Now lets take a look visually at these weights.

In [ ]:
def weight_plot(queen, rook, kernel, knn, base, geo="South America"):

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,15), dpi=200)
    fig.suptitle('{} with different weights'.format(geo), fontsize=24)
    plot_spatial_weights( queen, base,ax=ax1)
    plot_spatial_weights( rook, base, ax=ax2)
    plot_spatial_weights( kernel, base, ax=ax3)
    plot_spatial_weights( knn, base, ax=ax4)

    ax1.set_title("Queen")
    ax2.set_title("Rook")
    ax3.set_title("Kernel (Bandwith =20)")
    ax4.set_title("KNN (n = 4)")
    
    if geo == "South America":

        ax1.set_xlim(-90, -30)
        ax1.set_ylim(-60, 20)
        ax2.set_xlim(-90, -30)
        ax2.set_ylim(-60, 20)
        ax3.set_xlim(-90, -30)
        ax3.set_ylim(-60, 20)
        ax4.set_xlim(-90, -30)
        ax4.set_ylim(-60, 20)
    
    elif geo == "South-Eastern Asia":
        
        ax1.set_xlim(80, 160)
        ax1.set_ylim(-20, 25)
        ax2.set_xlim(80, 160)
        ax2.set_ylim(-20, 25)
        ax3.set_xlim(80, 160)
        ax3.set_ylim(-20, 25)
        ax4.set_xlim(80, 160)
        ax4.set_ylim(-20, 25)
        
    fig.tight_layout()    
    plt.show()
    
weight_plot(queen,rook, kernel, knn, df)

In [ ]:
weight_plot(queen, rook, kernel, knn, df, geo="South-Eastern Asia")

In [ ]:
non_planar = libpysal.weights.util.nonplanar_neighbors(queen, df)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10),dpi=80)
plot_spatial_weights(non_planar, df, ax=ax)
ax.set_xlim(80, 160)
ax.set_ylim(-20, 25)